# Semi-automatization of DRN and SFR sensitivity analysis/calibration

**Worflow**\
DRN
1. Load .drn file - *completed*
2. Change the conductance of the cells - *completed*
3. Write the .drn file - *completed*
4. Run the model - *in progress*
5. Read .cbb file
6. Store the outflow data of the first X cells
7. Put the process in a loop over a range of parameters

SFR
1. Load .sfr file - *in progress*
2. Change the hydraulic conductivity of the cells
3. Write the .sfr file
4. Run the model
5. Read streamflow.dat file
6. Store the flux data of cell at reach X
7. Put the process in a loop over a range of parameters

## Setup

In [1]:
import flopy
import flopy.utils.binaryfile as bf
import glob
import numpy as np
import os
import pandas as pd

#check if the below lines are still useful at the end
import sys
from pprint import pformat
from tempfile import TemporaryDirectory

In [2]:
# cwd = "C:/Users/user/OneDrive - Politecnico di Milano/hydrogeo-modelling/TEAM_Idrogeo/Tesi/Tesi_Ceola_Pirovano/Tesi_Pirovano/busca_drain_start"
cwd = os.getcwd()

In [3]:
def import_input_file(path):
    f = open(path).readlines()
    df = pd.DataFrame()
    for row in f[4:]:
        r = list(filter(None, row.split(' ')))
        df = pd.concat([df, pd.DataFrame(r).transpose()])
    df.columns = ['layer', 'row', 'column', 'stage', 'conductance', 'node']
    df.node = df.node.str.removesuffix('\n')
    df.layer = df.layer.astype('int')
    df.row = df.row.astype('int')
    df.column = df.column.astype('int')
    df.stage = df.stage.astype('float')
    df.conductance = df.conductance.astype('float')
    df.reset_index(inplace=True, drop = True)
    return df

## DRAIN

### Load .drn file

In [4]:
# Load .drn file
drn = import_input_file(os.path.join(cwd, 'test_files', 'busca_drain.drn'))
drn.head()

,layer,row,column,stage,conductance,node
0,1,46,51,124.838,0.012,0
1,1,47,51,124.835,0.018,0
2,1,48,51,124.832,0.006,0
3,1,48,52,124.830,0.024,0
4,1,49,52,124.824,0.018,0


In [5]:
# Load DRAIN cells characteristics
drn_sp = pd.read_csv(os.path.join(cwd, 'test_files', 'busca_drain_specifiche_celle.csv'))

In [14]:
# Compute the conductance of each cell for a given hydraulic conductivity
# Change the conductance column
k = 0.00001444444444444 #hydraulic conductivity
conductance_change = (k*drn_sp.Width*drn_sp.Length)/drn_sp.Thickness
drn.conductance = conductance_change
# drn

### Write .drn file using flopy

**bug found:** the layer gets put equal to 2 instead of 1
- the problem is not in the stress_period_data table passed to drn
- check stress_period_data.write_transient method
- couldn't figure it out, so found a way out: set drn.layer = 0
- drn_input_file_test_v4.drn works

In [8]:
modelpth = os.path.join(cwd, 'test_files')

# create the model class (only useful because it's needed by ModflowDrn class)
# this can be replaced by loading the actual model .nam file (not needed though)
mf = flopy.modflow.Modflow(
     "drn_test",
    model_ws = modelpth,
    exe_name = "mf2005",
)

In [9]:
# transform the drn structure in the flopy required format for "stress_period_data"
drn.layer = 0
stress_period_data = {0: drn.iloc[:, :-1].to_numpy().tolist()} #remove node column, not needed nor supported by flopy's ModflowDrn class

In [10]:
#generate the drn package inside the flopy class
ipakcb = 50 #code for cell-by-cell flow data storage
drain = flopy.modflow.ModflowDrn(mf, ipakcb=ipakcb, stress_period_data=stress_period_data,
                                 filenames=os.path.join(cwd, 'test_files', 'drn_input_file_test_v5.drn'))
drain.write_file(check = False)

### Test: Run the model

In [12]:
# trial: run from flopy

# load the existing model busca_drain
# https://flopy.readthedocs.io/en/3.3.2/source/flopy.modflow.mf.html

cwd_model = "d:/Claudia/MAURICE/test_models/"

mf = flopy.modflow.Modflow.load(
        os.path.join(cwd_model, 'drain_model_test', 'busca_drain.nam'),
        model_ws = os.path.join(cwd, 'test_files', 'model_test'),
        exe_name='MF2005',
        version = 'mf2005',
        verbose = False
        )

In [ ]:
# drn stress period data can be accessed by:
mf.drn.stress_period_data.data

In [ ]:
# this works
success, buff = flopy.mbase.run_model(
                exe_name = os.path.join(cwd_model, 'drain_model_test','MF2005.exe'),
                namefile = 'busca_drain.nam',
                model_ws = os.path.join(cwd_model, 'drain_model_test')
                )
if not success:
    raise Exception("MODFLOW did not terminate normally.")

### Read .cbb output file

class CellBudgetFile\
https://flopy.readthedocs.io/en/latest/source/flopy.utils.binaryfile.html

The outflow will have to be considered until cell:
- row 92, column 76 (last cell of first segment)
Check if any substantial change in outflow happen if considering:
- row 93, column 76 (first cell of second segment)

In [ ]:
#get drain values from cbb output
cbb = bf.CellBudgetFile(os.path.join(cwd_model, 'drain_model_test', 'busca_drain.cbb'))
drain = cbb.get_data(text = 'DRAINS')

# get all the cells until 92, 76 & sum the flux extracted from drain[0]
row_limit = 92
column_limit = 76
sum_drain = np.sum(drain[0][0, :row_limit+1, :column_limit+1])
sum_drain

### LOOP

In [ ]:
#Setup
#Import all necessary packages
import flopy
import flopy.utils.binaryfile as bf
import glob
import numpy as np
import os
import pandas as pd
import sys
from pprint import pformat
from tempfile import TemporaryDirectory

#Define working directory 
cwd = os.getcwd()   #where test files are stored (github)
cwd_model = "d:/Claudia/MAURICE/test_models/"  #where model files are stored (local)

# Function to load .drn file directly
def import_input_file(path):
    f = open(path).readlines()
    df = pd.DataFrame()
    for row in f[4:]:
        r = list(filter(None, row.split(' ')))
        df = pd.concat([df, pd.DataFrame(r).transpose()])
    df.columns = ['layer', 'row', 'column', 'stage', 'conductance', 'node']
    df.node = df.node.str.removesuffix('\n')
    df.layer = df.layer.astype('int')
    df.row = df.row.astype('int')
    df.column = df.column.astype('int')
    df.stage = df.stage.astype('float')
    df.conductance = df.conductance.astype('float')
    df.reset_index(inplace=True, drop = True)
    return df

# Load original .drn file, load DRAIN cells characteristics and
# transform the drn structure in flopy (required format for "stress_period_data")
drn = import_input_file(os.path.join(cwd, 'test_files', 'busca_drain.drn'))
drn_sp = pd.read_csv(os.path.join(cwd, 'test_files', 'busca_drain_specifiche_celle.csv'))
drn.layer = 0

# Create and load the model class (only useful because it's needed by ModflowDrn class)
# this can be replaced by loading the actual model .nam file (not needed though)
modelpth = os.path.join(cwd_model, 'drain_model_test') #path to model
# mf = flopy.modflow.Modflow(
#     "busca_drain",
#     model_ws = os.path.join(cwd_model, 'drain_model_test'),
#     exe_name = "mf2005",
# )
mf = flopy.modflow.Modflow.load(
    os.path.join(cwd_model, 'drain_model_test', 'busca_drain.nam'),
    model_ws = os.path.join(cwd_model, 'drain_model_test'),
    exe_name='MF2005',
    version = 'mf2005',
    verbose = False
    )
'''
START OF LOOP
'''
# Define limits of hydraulic conductivity range and number of iterations
ki = 0.01
kf = 0.000001
n = 10 #10 to test the code, then switch to 100
step = np.linspace(ki,kf,n)
inputs = []
outputs = []

ipakcb = 50 #code for cell-by-cell flow data storage

# Loop to change conductance in the drain package, run the model and extract cbb results
for i in range(n):
    k = step[i]
    #change conductance
    conductance_change = (k*drn_sp.Width*drn_sp.Length)/drn_sp.Thickness
    drn.conductance = conductance_change
    stress_period_data = {0: drn.iloc[:, :-1].to_numpy().tolist()} #remove node column, not needed nor supported by flopy's ModflowDrn class

    #generate the drn package inside the flopy class
    drain = flopy.modflow.ModflowDrn(mf, ipakcb=ipakcb, stress_period_data=stress_period_data,
                                 filenames=os.path.join(cwd_model, 'drain_model_test', 'busca_drain.drn'))
    drain.write_file(check = False)
    
    #run the model
    success, buff = mf.run_model(silent=False) #False to test the code, then switch to True
    if not success:
        raise Exception("MODFLOW did not terminate normally.")
    
    #get drain values from cbb output
    cbb = bf.CellBudgetFile(os.path.join(cwd_model, 'drain_model_test', 'busca_drain.cbb'))
    drain = cbb.get_data(text = 'DRAINS')

    # get all the cells until 92, 76 & sum the flux extracted from drain[0]
    row_limit = 92
    column_limit = 76
    sum_drain = np.sum(drain[0][0, :row_limit+1, :column_limit+1])

    # Append the input and output to lists
    inputs.append(k)
    outputs.append(sum_drain)

    #print(k)

#save to dataframe and export
df_results = pd.DataFrame({'k_value': inputs, 'drain_results': outputs})
df_results.to_excel(os.path.join(cwd, 'test_files','results.xlsx'))

c:\Users\user\anaconda3\envs\model\Lib\site-packages\flopy\mbase.py:637: UserWarning: Two packages of the same type, Replacing existing 'DRN' package.
  warn(


FloPy is using the following executable to run the model: C:\GWV8\MF2005.exe

                                  MODFLOW-2005     
    U.S. GEOLOGICAL SURVEY MODULAR FINITE-DIFFERENCE GROUND-WATER FLOW MODEL
                             Version 1.11.00 8/8/2013                        

 Using NAME file: busca_drain.nam 
 Run start date and time (yyyy/mm/dd hh:mm:ss): 2024/10/30  9:43:18

 Solving:  Stress period:     1    Time step:     1    Ground-Water Flow Eqn.
 Run end date and time (yyyy/mm/dd hh:mm:ss): 2024/10/30  9:43:18
 Elapsed run time:  0.165 Seconds

  Normal termination of simulation
FloPy is using the following executable to run the model: C:\GWV8\MF2005.exe

                                  MODFLOW-2005     
    U.S. GEOLOGICAL SURVEY MODULAR FINITE-DIFFERENCE GROUND-WATER FLOW MODEL
                             Version 1.11.00 8/8/2013                        

 Using NAME file: busca_drain.nam 
 Run start date and time (yyyy/mm/dd hh:mm:ss): 2024/10/30  9:43:18

 Solvi

c:\Users\user\anaconda3\envs\model\Lib\site-packages\flopy\mbase.py:628: UserWarning: Unit 21 of package DRN already in use.
  warn(
c:\Users\user\anaconda3\envs\model\Lib\site-packages\flopy\mbase.py:637: UserWarning: Two packages of the same type, Replacing existing 'DRN' package.
  warn(
c:\Users\user\anaconda3\envs\model\Lib\site-packages\flopy\mbase.py:628: UserWarning: Unit 21 of package DRN already in use.
  warn(
c:\Users\user\anaconda3\envs\model\Lib\site-packages\flopy\mbase.py:637: UserWarning: Two packages of the same type, Replacing existing 'DRN' package.
  warn(



                                  MODFLOW-2005     
    U.S. GEOLOGICAL SURVEY MODULAR FINITE-DIFFERENCE GROUND-WATER FLOW MODEL
                             Version 1.11.00 8/8/2013                        

 Using NAME file: busca_drain.nam 
 Run start date and time (yyyy/mm/dd hh:mm:ss): 2024/10/30  9:43:18

 Solving:  Stress period:     1    Time step:     1    Ground-Water Flow Eqn.
 Run end date and time (yyyy/mm/dd hh:mm:ss): 2024/10/30  9:43:19
 Elapsed run time:  0.162 Seconds

  Normal termination of simulation
FloPy is using the following executable to run the model: C:\GWV8\MF2005.exe

                                  MODFLOW-2005     
    U.S. GEOLOGICAL SURVEY MODULAR FINITE-DIFFERENCE GROUND-WATER FLOW MODEL
                             Version 1.11.00 8/8/2013                        

 Using NAME file: busca_drain.nam 
 Run start date and time (yyyy/mm/dd hh:mm:ss): 2024/10/30  9:43:19



c:\Users\user\anaconda3\envs\model\Lib\site-packages\flopy\mbase.py:628: UserWarning: Unit 21 of package DRN already in use.
  warn(
c:\Users\user\anaconda3\envs\model\Lib\site-packages\flopy\mbase.py:637: UserWarning: Two packages of the same type, Replacing existing 'DRN' package.
  warn(
c:\Users\user\anaconda3\envs\model\Lib\site-packages\flopy\mbase.py:628: UserWarning: Unit 21 of package DRN already in use.
  warn(
c:\Users\user\anaconda3\envs\model\Lib\site-packages\flopy\mbase.py:637: UserWarning: Two packages of the same type, Replacing existing 'DRN' package.
  warn(


...

FloPy is using the following executable to run the model: C:\GWV8\MF2005.exe

                                  MODFLOW-2005     
    U.S. GEOLOGICAL SURVEY MODULAR FINITE-DIFFERENCE GROUND-WATER FLOW MODEL
                             Version 1.11.00 8/8/2013                        

 Using NAME file: busca_drain.nam 
 Run start date and time (yyyy/mm/dd hh:mm:ss): 2024/10/30  9:43:19

 Solving:  Stress period:     1    Time step:     1    Ground-Water Flow Eqn.
 Run end date and time (yyyy/mm/dd hh:mm:ss): 2024/10/30  9:43:19
 Elapsed run time:  0.170 Seconds

  Normal termination of simulation
FloPy is using the following executable to run the model: C:\GWV8\MF2005.exe


c:\Users\user\anaconda3\envs\model\Lib\site-packages\flopy\mbase.py:628: UserWarning: Unit 21 of package DRN already in use.
  warn(
c:\Users\user\anaconda3\envs\model\Lib\site-packages\flopy\mbase.py:637: UserWarning: Two packages of the same type, Replacing existing 'DRN' package.
  warn(
c:\Users\user\anaconda3\envs\model\Lib\site-packages\flopy\mbase.py:628: UserWarning: Unit 21 of package DRN already in use.
  warn(
c:\Users\user\anaconda3\envs\model\Lib\site-packages\flopy\mbase.py:637: UserWarning: Two packages of the same type, Replacing existing 'DRN' package.
  warn(



                                  MODFLOW-2005     
    U.S. GEOLOGICAL SURVEY MODULAR FINITE-DIFFERENCE GROUND-WATER FLOW MODEL
                             Version 1.11.00 8/8/2013                        

 Using NAME file: busca_drain.nam 
 Run start date and time (yyyy/mm/dd hh:mm:ss): 2024/10/30  9:43:19

 Solving:  Stress period:     1    Time step:     1    Ground-Water Flow Eqn.
 Run end date and time (yyyy/mm/dd hh:mm:ss): 2024/10/30  9:43:20
 Elapsed run time:  0.178 Seconds

  Normal termination of simulation
FloPy is using the following executable to run the model: C:\GWV8\MF2005.exe

                                  MODFLOW-2005     
    U.S. GEOLOGICAL SURVEY MODULAR FINITE-DIFFERENCE GROUND-WATER FLOW MODEL
                             Version 1.11.00 8/8/2013                        

 Using NAME file: busca_drain.nam 
 Run start date and time (yyyy/mm/dd hh:mm:ss): 2024/10/30  9:43:20

 Solving:  Stress period:     1    Time step:     1    Ground-Water Flow Eqn.
 Run 

c:\Users\user\anaconda3\envs\model\Lib\site-packages\flopy\mbase.py:628: UserWarning: Unit 21 of package DRN already in use.
  warn(
c:\Users\user\anaconda3\envs\model\Lib\site-packages\flopy\mbase.py:637: UserWarning: Two packages of the same type, Replacing existing 'DRN' package.
  warn(


 Run end date and time (yyyy/mm/dd hh:mm:ss): 2024/10/30  9:43:20
 Elapsed run time:  0.149 Seconds

  Normal termination of simulation


## SFR

### Test: load .sfr file

In [ ]:
#the file is multi structured:
# - the above section has a structure similar to .drn
# - in the last section, the structure is different: there is the info about the segments
# - width and manning coefficient are defined in the segment section
# - length, slope, bed thickness and bed hydraulic conductivity are defined in the first section
# try to use flopy to load the model and then modify the sfr parameters directly from there

# another possibility:
# - export sfr as .txt from the model
# - read its characteristics here
# - create a tool flopy model
# - add sfr to the tool model
# - generate the input file

In [ ]:
sfr = open(os.path.join(cwd, 'test_files', 'busca_sfr2.sfr')).readlines()
sfr

In [ ]:
#error when loading the sfr package
mf = flopy.modflow.Modflow.load(
        f = os.path.join(cwd, 'test_files', 'sfr_model_test', 'busca_sfr2.nam'),
        # model_ws = os.path.join(cwd, 'test_files', 'sfr_model_test'),
        # exe_name='MF2005',
        # version = 'mf2005',
        verbose = True
        )

In [29]:
#this works now, because we changed the input file structure to SFR1 from GWV
#next step:
# - identify the parameters inside the sfr class
# - modify the k
# - save the new .sfr file
m = flopy.modflow.Modflow()
f = os.path.join(cwd_model, 'sfr_model_test', 'test_sfr1_input', 'busca_sfr2_tp.sfr')

sfr = flopy.modflow.ModflowSfr2.load(f, m, gwt =True)

In [ ]:
sfr

In [ ]:
sfr.reach_data

### Store the flow at reach x

Consider the outward flow of these two cells:
- reach 72, segment 1
- reach 1, segment 2

## Other tests

### Test: write .drn file autonomously without using flopy

Riprovare con il pacchetto di flopy usando un numpy array invecedel dataframe


Da qui stavo scrivendo write_file per scrivermi i dati
https://flopy.readthedocs.io/en/latest/_modules/flopy/modflow/mfdrn.html#ModflowDrn.write_file

stavo indagando come flopy gestisce lo stress_period_data (MFils o qualcosa)

In [ ]:
ipakcb = 1
line = f"{2:10d}{ipakcb:10d}"
line

In [ ]:
heading = '# DRN package for MODFLOW-2005 generated by paolchol'
n_drain_cells = drn.shape[0]
ipakcb = 50


with open(os.path.join(cwd, 'drn_input_file_test.drn'), 'w') as file:
    file.write(f"{heading}\n")
    line = f"{n_drain_cells:10d}{ipakcb:10d}"
    # for opt in self.options:
        # line += " " + str(opt)
    line += "\n"
    file.write(line)

    # for row in drn.iterrows():
    #     file.writelines(row)